In [1]:
import pandas as pd
import requests

from register_new_users import MemberManager

In [2]:
manager = MemberManager()

In [3]:
responses = manager.get_responses("register")
responses["First_name"] = responses["First Name"]
responses["Last_name"] = responses["Last Name"]
responses = responses.drop(["First Name", "Last Name"], axis=1)
responses.head(3)

,URL,First_name,Last_name
ACYDBNhqD4n2G_ao89Rr0s7xGPei9VJLRQFDx0jY_S5yBNW46DdrgtaSAwh9F26u6BjW148,http://localhost/exchange_token?state=&code=12...,Ben,Hoskings
ACYDBNgaQAoZxX14iD5yxQV2wlhMsKsINrW62nIEGLQC8rCZSrUGobgZBkM89rSSikuaxAs,http://localhost/exchange_token?state=&code=af...,Benjamina,Hos


In [12]:
responses = manager.get_responses("session_input")
print(responses.shape)
responses.head(10)

(6, 4)


,name,date,other_info,rep_info
ACYDBNit2IepGblmkwIZHVbt7HAVpp8XKieP7g9nVwS8wNfLdr4VBCrU3pmCJnqtrWe1Vzk,Sam Marshall,2024-08-20,grass,lap time dist pace \n1 1 1:01 0.16 ...
ACYDBNhcg1ajR_6KRY-xBEnwMcKrpplAFsd3r2eqbXjewrRzsBvDJLeHC473H0iUA0-6OFc,Sam Marshall,2024-08-22,"Mile,K,Mile,K",lap time dist pace \n1 1 6:07 0.99...
ACYDBNhVeZE0tvz0axC9ApMsFcBhaygFtKKf25e23u0WZtKJ3Y7K9bVt9gsgwa9B_Zwjb7g,Jodie,2024-08-27,Two reps but stopped after 1 half little pain ...,"60, 80, 100, 120"
ACYDBNgOxGsejQSR7emEBhLTy5ZaUz_SP4RlZte-gng0uBBlEVUcD2BoWdqAem_qaEr8eew,Sam Marshall,2024-08-27,3x(5x300m),lap time dist pace \n 1 1 1:00 0.1...
ACYDBNi41iqyuYteIc14cOOgNxBY9AUkbWd2XtTK_eQI1VdOE7TLZiuJVPIUWDdhp_wfpR4,Ant,2024-08-27,None,3 sets of 300x5
ACYDBNiUZeJFg4EfC1C09d5FFqkgVh-bf41W0VeJNNUzSXZ8EA1YfdPI0AjafO2cta9x3HA,Sam Marshall,2024-08-24,parkrun,18:44


In [5]:
access_data_path = "access_tokens.tsv"

access_data = pd.read_csv(access_data_path, sep="\t")
access_data.set_index("ID", inplace=True)
access_data.head(3)

,First_name,Last_name,Access_token,Expires_at,Refresh_token
ID,,,,,
963223.0,Neil,Hoskings,69d8c42f38dcecbbfb3947f052d54c1ca049f0c8,1.724886e+09,0aa21162002e540df4dae47840ddc42f17745c75
98588012.0,Becky,Hoskings,6bd688ab1a58a50794aed5d5c7eee236cb0e0d43,1.724886e+09,23d2c1b7a49bc26cd22afb819b3a19d6db7ffcf3
38781770.0,Clark,Roberts,675f08a6a61efa4ec10e60a28677f3b560d74861,1.724886e+09,ab54a8bd816db93297ef7cf7022ffa76e6efc36b


In [6]:
def get_access_tokens(code):
    response = requests.post(
        url='https://www.strava.com/oauth/token',
        data={
            'client_id': 111130,
            'client_secret': 'b7fca9288af24aaa2f86f1b82f70f2baa54daeb2',
            'code': code,
            'grant_type': 'authorization_code'
        }
    )

    tokens = response.json()

    try:
        athlete_tokens = {
            "ID": tokens["athlete"]["id"],
            "Access_token": tokens["access_token"],
            "Refresh_token": tokens["refresh_token"]
        }

    except:
        athlete_tokens = tokens

    return athlete_tokens

In [7]:
# tokens = {"ID": 38803221, "Access_token": 'cee165ac6b8f4c4db712e81accbfafeed0cc5f7a', "Refresh_token": '840befc02a4dca632ce7439fc082942c282f4e70'}

for response_id in responses.index:
    response = responses.loc[response_id]
    # print(response.to_dict())
    try:
        parts = response["URL"].split("&")
        code = parts[1][5:]
        tokens = get_access_tokens(code)
        if tokens:
            if "message" in tokens.keys():
                print("link extraction failed")
            else:
                responses.loc[response_id, "ID"] = tokens["ID"]
                responses.loc[response_id, "Refresh_token"] = tokens["Refresh_token"]
                responses.loc[response_id, "Access_token"] = tokens["Access_token"]
                responses.loc[response_id, "Access_token"] = tokens["Access_token"]
                responses = responses.loc[:, ~responses.columns.str.contains('^Unnamed')]
                
    except Exception as e:
        # raise e
        print("link extraction failed")
        
if (responses.columns.str.contains('ID')).any():
    responses = responses.set_index("ID")
else:
    responses = None

if responses is not None:    
    responses.head()

link extraction failed
link extraction failed
link extraction failed
link extraction failed
link extraction failed
link extraction failed


In [8]:
access_data = pd.concat([access_data, responses], join='inner')
access_data = access_data.loc[access_data.index.dropna()]
# access_data.join(responses, on="ID", how="left")
access_data.head(10)

,First_name,Last_name,Access_token,Expires_at,Refresh_token
ID,,,,,
963223.0,Neil,Hoskings,69d8c42f38dcecbbfb3947f052d54c1ca049f0c8,1.724886e+09,0aa21162002e540df4dae47840ddc42f17745c75
98588012.0,Becky,Hoskings,6bd688ab1a58a50794aed5d5c7eee236cb0e0d43,1.724886e+09,23d2c1b7a49bc26cd22afb819b3a19d6db7ffcf3
38781770.0,Clark,Roberts,675f08a6a61efa4ec10e60a28677f3b560d74861,1.724886e+09,ab54a8bd816db93297ef7cf7022ffa76e6efc36b
33370063.0,Tom,Beesley,548f50ee7b152d2ed5bd52e39dc89fd19182de6b,1.724886e+09,5a14aa37c6b245c36dccdb5b5c4467f1996d327c
71665908.0,Sophie,Hurst,8764ffc2a9d446d8ea8c08d5bae82672bbcd94c2,1.724886e+09,bbe9aa060ed7423b16777886520dfc554bd6052e
14188705.0,Wais,Yip,a374fb131384cca69d3b1161a2898db3f1f2c192,1.724886e+09,995a9f58911040bf48924824e8e9bd8e843b5097
38807221.0,Ben,Hoskings,1229671ff5af8b4490676071b794f384479df512,1.724886e+09,840befc02a4dca632ce7439fc082942c282f4e70


In [9]:

access_data.to_csv("access_data_update.tsv", sep="\t", index=True)

In [10]:
import requests
import json
url = "https://eu-west-2.aws.data.mongodb-api.com/app/data-izribar/endpoint/data/v1/action/findOne"

payload = json.dumps({
    "collection": "athlete_data",
    "database": "godiva_data",
    "dataSource": "Cluster0"
})

headers = {
  'Content-Type': 'application/json',
  'Access-Control-Request-Headers': '*',
  'api-key': 'b8bRma7UNr1eGbaQjBb2aftslsjCe3SGltB4acl8o5jC0hrNstH2O1zmaV8QT8zx',
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"document":{"_id":"66585b3e1941040e53220a27","first_name":"Ben","last_name":"Hoskings","dob":"27/07/2002","strava_id":38807221,"group":"REPS"}}
